# <center> Capstone Neighborhood Clustering.</center>

In [1]:
#importing the neccesary libraries
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [2]:
#getting the wiki url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# we load the url and use soup to get all the values in the table class = wikitable
web = requests.get(url).text
soup = bs(web,'lxml')
wiki = soup.find("table",class_='wikitable')

In [4]:
#we need now to iterate and find all values between tr and td, we use strip to remove the tr and td
data_clean=[]
for tr in wiki.find_all('tr'):
    for td in tr.find_all('td'):
        data_clean.append((td.find_all(text=True)[0].strip()))

In [5]:
# This gets us a list of all the values we need, but now, in order to transform into a dataframe, we need to group them every 3, so we iterate again 
data_final=[]
for i in range(0, len(data_clean), 3):
  data_final.append([data_clean[i],data_clean[i+1],data_clean[i+2]])

In [6]:
#we convert the list to a df
data_fr=pd.DataFrame(data_final)

In [7]:
#add the column headers
data_fr.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [8]:
#we check how we are doing
data_fr.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [9]:
#we remove the not assigned rows on borough
data_fr = data_fr.loc[data_fr['Borough'] != 'Not assigned']

In [10]:
# replacing the not assigned value with the borough value on neiighborhood column
data_fr['Neighborhood'].replace('Not assigned', data_fr['Borough'], inplace=True)
   

In [11]:
#we check, notice than Queens Park on the Neighborhood colum is no longer 'Not assigned' and correctly has borough value
data_fr.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [12]:
#grouping first by postal code and then by borough, to join the last value
asd = data_fr.groupby(['PostalCode', 'Borough']).agg(', '.join)

In [13]:
#resetting the indexes, and defining the df variable to the one we are goign to use.
df = asd.reset_index()

In [14]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [15]:
#final shape of the dataframe
df.shape

(103, 3)